In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os


def add_to_syspath(path: str):
    for p in sys.path:
        if p == path:
            return
    sys.path.append(path)


# upper folder
add_to_syspath('../')

In [3]:
from photoprism_client import ApiClient
from photoprism_client import Configuration
from photoprism_client import AlbumsApi
from photoprism_client import PhotosApi
from photoprism_client import FacesApi

# Custom Authentification
- create config.json file
    ```
    {
        "host": "https://hostname",
        "username": "USER",
        "my_ca": "ca.crt",
        "auth_key": "123456-123456-123456-123456"
    }
    ```

In [ ]:
import json
try:
    with open('config.json') as f:
        config = json.load(f)
        host = config['host']
        username = config['username']
        my_ca = config['my_ca']
        auth_header = "X-Auth-Token"
        auth_key = config['auth_key']

    print('using custom config')
except Exception as e:
    # same as using photoprism_client.ApiClient()
    host = 'https://demo.photoprism.app'
    username = 'demo'
    my_ca = None
    auth_header = "X-Auth-Token"
    auth_key = '123456-123456-123456-123456'
    print('using default config')

In [5]:
import photoprism_client
from photoprism_client.rest import ApiException
from pprint import pprint

# Defining the host is optional and defaults to http://demo.photoprism.app
# See configuration.py for a list of all supported configuration parameters.
configuration = photoprism_client.Configuration(
    host=host,
    username=username,
    ssl_ca_cert=my_ca
)


In [ ]:
# Enter a context with an instance of the API client
with photoprism_client.ApiClient(configuration,
                                 header_name=auth_header,
                                 header_value=auth_key) as api_client:
    # Create an instance of the API class
    albums = photoprism_client.AlbumsApi(api_client)

    for album in albums.search_albums(count=5):
        print(album.uid, album.path)

In [7]:

albums_by_path = albums.search_albums(count=100000)


In [ ]:
import os
group = {}

for photo in albums_by_path:
    paths = photo.path.split('/')

    try:
        if len(paths) > 1:
            group[paths[0]] += [paths[1]]

    except:
        if len(paths) > 1:
            group[paths[0]] = [paths[1]]
        else:
            group[paths[0]] = []

# print folder structure of albums
for k in group:
    print(k)
    print(' - '+'\n - '.join(set(group[k])))

In [ ]:
# Enter a context with an instance of the API client
with photoprism_client.ApiClient(configuration,
                                 header_name=auth_header,
                                 header_value=auth_key) as api_client:
    # Create an instance of the API class
    api_instance = photoprism_client.PhotosApi(api_client)
    photos = photoprism_client.FormSelection()  # FormSelection | Photo Selection

    for photo in api_instance.search_photos(count=5, quality=1,
                                            q=f'{username} country:"jp|us"' if not username == 'demo' else None
                                            ):
        print(photo.file_name, photo.path, photo.uid)

In [ ]:


# Enter a context with an instance of the API client
with photoprism_client.ApiClient(configuration,
                                 header_name=auth_header,
                                 header_value=auth_key) as api_client:
    # Create an instance of the API class
    api_instance = photoprism_client.AlbumsApi(api_client)
    uid = 'uid_does_not_exist'  # str | Album UID
    photos = photoprism_client.FormSelection()  # FormSelection | Photo Selection
    photos.photos = [
        'pskk05176wjvrd9p', 'pskk0o2959k7pixn',
        'pslit7ifzljmbfes',
        ]
    album_form = photoprism_client.FormAlbum()
    album_form.title = 'api_test'
    try:
        album = api_instance.create_album(album_form)
        uid = album.uid
        # adds photos to an album
        api_response = api_instance.add_photos_to_album(uid, photos)
        print("The response of AlbumsApi->add_photos_to_album:\n")
        pprint(api_response)
    except ApiException as e:
        print("Exception when calling AlbumsApi->add_photos_to_album: %s\n" % e)